# Evaluating combo SAEs

In [ ]:
from main import *

## Max activating features

## Linearity of SAE sizes

In [3]:
# Train a larger regular SAE, hidden size 32 * 768 = 24576
import torch
from main_regular import main_regular_sae  # Adjust the import based on the file location

# Manually specify the arguments
layer = 9
model_type = 'gated'
n_batches = 5000
l1_coefficient = 0.5
batch_size = 4
lr = 0.001
projection_up = 32
repo_name = "regular-sae"

# Call the function with specified arguments
main_regular_sae(layer, model_type, n_batches, l1_coefficient, batch_size, lr, projection_up, repo_name)

Running on cpu...


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


dead_neurons,▁██
l0_loss,█▁▁
recon_loss,█▄▁
test_loss,█▁▁
dead_neurons,97.51383
l0_loss,0.15664
recon_loss,45.29677
test_loss,90.6025
training_type,regular_sae


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Training Batches:  48%|████▊     | 2375/5000 [25:47<1:05:47,  1.50s/it]

Batch 2375 Test Loss 46.1468 | Reconstruction Error 19.4425 | L0 Loss 44.7896 | Dead Neurons: 98.86%


Training Batches:  48%|████▊     | 2400/5000 [26:03<1:08:34,  1.58s/it]

Batch 2400 Test Loss 43.9954 | Reconstruction Error 18.6521 | L0 Loss 42.5916 | Dead Neurons: 98.48%


Training Batches:  48%|████▊     | 2412/5000 [26:10<22:52,  1.89it/s]  

In [ ]:
# Train a smaller regular SAE, hidden size 16_000

In [ ]:
# Train a smaller error SAE, hidden size 16_000
layer = 9
model_type = 'gated'
n_epochs = 100
l1_coefficient = 3e-4
batch_size = 2048
lr = 0.001
projection_up = 16
repo_name = "error-saes"

error_sae = main(layer, model_type, n_epochs, l1_coefficient, projection_up, batch_size, lr, repo_name, return_model=True, save_model=False)